In [30]:
%reload_ext autoreload
%autoreload 2

from pytorch_lightning import seed_everything
from src.acnets.deep import WaveletModel, MaskedModel
from src.acnets.datasets import LEMONDataModule, Julia2018DataModule
from src.acnets.deep.gan import GAN

for n_embeddings in [32]:

    seed_everything(42)
    input_type = 'cconn_networks'

    pretrain_datamodule = LEMONDataModule(
        atlas='dosenbach2010', kind='partial correlation',
        aggregation_strategy=input_type,
        test_ratio=.1, val_ratio=.05,
        segment_length=124,
        n_subjects=215, batch_size=31, shuffle=True)
    pretrain_datamodule.setup()

    finetune_datamodule = Julia2018DataModule(
        atlas='dosenbach2010', kind='partial correlation',
        aggregation_strategy=input_type,
        segment_length=124,
        test_ratio=.25, batch_size=8, shuffle=True)
    finetune_datamodule.setup()

    n_features = finetune_datamodule.train[0][0].shape[-1]

    # model = MaskedModel(n_features * n_features, n_embeddings, input_type=input_type)
    model = GAN(n_features * n_features, n_embeddings, n_classes=2)

    trainer = model.fit(pretrain_datamodule, max_epochs=10000, phase='pretrain')
    tuner = model.fit(finetune_datamodule, max_epochs=10000, phase='finetune')

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


ckpt_id: 628d68


/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /home/morteza/workspace/ACNets/checkpoints/GAN_628d68 exists and is not empty.
Restoring states from the checkpoint path at /home/morteza/workspace/ACNets/checkpoints/GAN_628d68/last.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 53.5 K
1 | discriminator | Discriminator | 50.8 K
------------------------------------------------
104 K     Trainable params
0         Non-trainable params
104 K     Total params
0.417     Total estimated model params size (MB)
Restored all states from the checkpoint at /home/morteza/workspace/ACNets/checkpoints/GAN_628d68/last.ckpt
`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
H